# APSIM Database Files Conversion

This notebook provides tools and workflows for converting and processing APSIM database files.

In [47]:
# Import required libraries
import sqlite3
import json
import os
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import pandas as pd

print("✓ Imports successful")
print("Note: pandas is required for CSV conversion")
print("Install with: pip install pandas")

✓ Imports successful
Note: pandas is required for CSV conversion
Install with: pip install pandas


## Database Schema

Each APSIM database file contains two main tables with simulation outputs:

### Report Table (End-of-Season Information)

The Report table contains annual/seasonal summary data with the following fields:

| Field | Description |
|-------|-------------|
| **Year** | Simulation year |
| **Latitude** | Geographic latitude |
| **Longitude** | Geographic longitude |
| **Cultivar** | Crop cultivar/variety identifier |
| **CropSurvived** | Logical output indicating if the crop survived the season. If `False` (or 0), the model 'killed' the crop. Note: zero yield could occur for other reasons than a dead crop. |
| **SowingDate** | Date when crop was sown |
| **CurrentPhenologyStage** | Final phenology stage reached |
| **FloweringDaysAfterSowing** | Number of days from sowing to flowering |
| **MaturityDaysAfterSowing** | Number of days from sowing to maturity |
| **HarvestDate** | Date when crop was harvested |
| **Yield** | Dry weight yield in kg/ha (Note: column name is "Yield", not "DryYield") |
| **WetYield** | Wet weight yield in kg/ha (grain moisture of 12%) |
| **FrostHeatYield** | Yield in kg/ha affected by frost and heat extreme events. If this equals Yield, then no frost or heat effect occurred. |
| **GrainProtein** | Grain protein percentage (will always be high with UNLIMITED N in these simulations) |
| **InCropRain** | Rainfall (mm) between sowing and harvest day |
| **FallowRain** | Rainfall (mm) before sowing |
| **SowingPAW** | Plant Available Water (mm) at sowing time |
| **HarvestPAW** | Plant Available Water (mm) at harvest time |
| **PAWC** | Soil Plant Available Water Capacity (mm) |
| **CumulativeFrost** | Percentage frost impact on yield |
| **CumulativeHeat** | Percentage heat impact on yield |
| **NumFrostEvents** | Number of frost events during the season |
| **NumHeatEvents** | Number of heat events during the season |

### Daily Table (Daily Time Series)

The Daily table contains daily outputs from the simulations with the following fields:

| Field | Description |
|-------|-------------|
| **Date** | Date of the simulation day |
| **Year** | Simulation year |
| **Latitude** | Geographic latitude |
| **Longitude** | Geographic longitude |
| **Cultivar** | Crop cultivar/variety identifier |
| **CropSurvived** | Logical output indicating if the crop survived. If `False` (or 0), the model 'killed' the crop. Note: zero yield could occur for other reasons than a dead crop. |
| **SowingDate** | Date when crop was sown |
| **CurrentPhenologyStage** | Current phenology stage on this day |
| **DryYield** | Dry weight yield in kg/ha (cumulative) |
| **WetYield** | Wet weight yield in kg/ha, considering grain moisture of 12% (cumulative) |
| **FrostHeatYield** | Yield in kg/ha affected by frost and heat extreme events. If this equals DryYield, then no frost or heat effect occurred. |
| **GrainProtein** | Grain protein percentage (will always be high with UNLIMITED N in these simulations) |
| **CumulativeFrost** | Percentage frost impact on yield (cumulative) |
| **CumulativeHeat** | Percentage heat impact on yield (cumulative) |
| **RadiationsFactor** | Radiation factor for the crop (0-1), where 1 = no stress (Note: column name is "RadiationsFactor" with an 's', not "RadiationFactor") |
| **WaterFactor** | Water factor (0-1), where 1 = no water stress |
| **NitrogenFactor** | Nitrogen factor (0-1), where 1 = no nitrogen stress |
| **VapourDeficitFactor** | Vapour deficit factor (0-1), where 1 = no vapour deficit stress |

### Additional Tables

The database may also contain metadata tables:
- **\_Simulations**: Simulation metadata
- **\_Checkpoints**: Checkpoint information
- **\_Messages**: Warnings, errors, and informational messages
- **\_Units**: Unit definitions
- **\_InitialConditions**: Initial soil conditions

In [48]:
# Configuration
BASE_DIR = r"C:\Users\ibian\Desktop\ClimAdapt"

# Option 1: Specify a specific database file (set these values)
FARM_NAME = "Anameka"  # e.g., "Anameka"
COORDINATE = "-31.45_117.55"  # e.g., "-31.45_117.55"
DB_FILE_NAME = "ClimAdapt_Wheat_neg31.45_117.55_585_calibrated"  # include the".db"

# Option 2: Leave DB_FILE_NAME = None to discover all database files

# Set your base directory here (should contain farm name folders)
base_directory = Path(BASE_DIR)

print("=" * 80)
print("Configuration")
print("=" * 80)
print(f"Base directory: {base_directory}")
print(f"Directory exists: {base_directory.exists()}")

if DB_FILE_NAME:
    # Specific file mode
    print(f"\nMode: SPECIFIC FILE")
    print(f"Farm name: {FARM_NAME}")
    print(f"Coordinate: {COORDINATE}")
    print(f"Database file: {DB_FILE_NAME}")
    
    # Construct the full path: base_directory / FARM_NAME / COORDINATE_APSIM / DB_FILE_NAME
    COORDINATE_APSIM = f"{COORDINATE}_APSIM"  # e.g., "-31.45_117.55_APSIM"
    specific_db_path = base_directory / FARM_NAME / COORDINATE_APSIM / DB_FILE_NAME
    print(f"\nFull path: {specific_db_path}")
    print(f"File exists: {specific_db_path.exists()}")
    
    # Set mode flag
    USE_SPECIFIC_FILE = True
    target_db_file = specific_db_path if specific_db_path.exists() else None
else:
    # Discovery mode
    print(f"\nMode: DISCOVER ALL FILES")
    print("Will search for all .db files in the nested structure")
    USE_SPECIFIC_FILE = False
    target_db_file = None

Configuration
Base directory: C:\Users\ibian\Desktop\ClimAdapt
Directory exists: True

Mode: SPECIFIC FILE
Farm name: Anameka
Coordinate: -31.45_117.55
Database file: ClimAdapt_Wheat_neg31.45_117.55_585_calibrated

Full path: C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.45_117.55_APSIM\ClimAdapt_Wheat_neg31.45_117.55_585_calibrated
File exists: False


## Database File Discovery

Find database file(s) based on configuration:
- **Specific file mode**: Uses the configured `FARM_NAME`, `COORDINATE`, and `DB_FILE_NAME`
- **Discovery mode**: Finds all `.db` files in the nested structure: `{base_path}/{farm_name}/{coordinate}_APSIM/*.db` (e.g., `{base_path}/{farm_name}/-31.45_117.55_APSIM/*.db`)

In [49]:
# Find database file(s) based on configuration
db_files_by_location = {}
total_files = 0
all_db_files = []

if USE_SPECIFIC_FILE:
    # Specific file mode
    if target_db_file and target_db_file.exists():
        print("=" * 80)
        print("Specific File Mode")
        print("=" * 80)
        print(f"\nUsing configured database file:")
        print(f"  File: {target_db_file.name}")
        print(f"  Farm: {FARM_NAME}")
        print(f"  Coordinate: {COORDINATE}")
        print(f"  Full Path: {target_db_file}\n")
        
        # Create structure similar to discovery mode
        location_key = f"{FARM_NAME}/{COORDINATE}"
        db_files_by_location[location_key] = {
            'farm_name': FARM_NAME,
            'coordinate': COORDINATE,
            'folder_path': str(target_db_file.parent),
            'db_files': [target_db_file]
        }
        
        all_db_files.append({
            'file_path': target_db_file,
            'farm_name': FARM_NAME,
            'coordinate': COORDINATE,
            'folder_path': str(target_db_file.parent),
            'file_name': target_db_file.name
        })
        
        total_files = 1
        print(f"✓ Database file loaded successfully")
    else:
        print("=" * 80)
        print("Error: Specific File Mode")
        print("=" * 80)
        if target_db_file:
            print(f"\n✗ Database file not found: {target_db_file}")
        else:
            print(f"\n✗ Invalid path configuration")
        print(f"\nPlease check:")
        print(f"  - Base directory: {base_directory} (exists: {base_directory.exists()})")
        print(f"  - Farm name: {FARM_NAME}")
        print(f"  - Coordinate: {COORDINATE}")
        print(f"  - Database file name: {DB_FILE_NAME}")
        
else:
    # Discovery mode - find all .db files
    if base_directory.exists():
        print("=" * 80)
        print("Discovery Mode - Finding All Database Files")
        print("=" * 80)
        
        # Iterate through farm name folders
        for farm_folder in base_directory.iterdir():
            if not farm_folder.is_dir():
                continue
            
            farm_name = farm_folder.name
            
            # Iterate through coordinate_APSIM folders within each farm
            for coord_apsim_folder in farm_folder.iterdir():
                if not coord_apsim_folder.is_dir():
                    continue
                
                folder_name = coord_apsim_folder.name
                
                # Check if folder name ends with "_APSIM"
                if folder_name.endswith("_APSIM"):
                    # Extract coordinate from folder name (remove "_APSIM" suffix)
                    coordinate = folder_name[:-6]  # Remove "_APSIM" (6 characters)
                    
                    # Find all .db files directly in this folder
                    db_files = list(coord_apsim_folder.glob("*.db"))
                    
                    if db_files:
                        location_key = f"{farm_name}/{coordinate}"
                        db_files_by_location[location_key] = {
                            'farm_name': farm_name,
                            'coordinate': coordinate,
                            'folder_path': str(coord_apsim_folder),
                            'db_files': db_files
                        }
                        total_files += len(db_files)
        
        # Display results
        if total_files > 0:
            print(f"\nFound {total_files} database file(s) across {len(db_files_by_location)} location(s):\n")
            
            for location_key, location_data in db_files_by_location.items():
                print(f"📍 {location_key}")
                print(f"   Folder: {location_data['folder_path']}")
                print(f"   Files ({len(location_data['db_files'])}):")
                for db_file in location_data['db_files']:
                    print(f"     - {db_file.name}")
                print()
            
            # Create a flat list of all db files with metadata
            for location_data in db_files_by_location.values():
                for db_file in location_data['db_files']:
                    all_db_files.append({
                        'file_path': db_file,
                        'farm_name': location_data['farm_name'],
                        'coordinate': location_data['coordinate'],
                        'folder_path': location_data['folder_path'],
                        'file_name': db_file.name
                    })
            
            print(f"Total: {total_files} database file(s) found")
        else:
            print(f"\nNo database files found in the structure.")
            print(f"Checked: {base_directory}")
    
    else:
        print("=" * 80)
        print("Error: Base Directory Not Found")
        print("=" * 80)
        print(f"\n✗ Base directory not found: {base_directory}")
        print(f"Please check the BASE_DIR configuration.")

print("\n" + "=" * 80)

Error: Specific File Mode

✗ Invalid path configuration

Please check:
  - Base directory: C:\Users\ibian\Desktop\ClimAdapt (exists: True)
  - Farm name: Anameka
  - Coordinate: -31.45_117.55
  - Database file name: ClimAdapt_Wheat_neg31.45_117.55_585_calibrated



## Selected Database File

Access the configured or selected database file for processing.

In [50]:
# Selected database file for processing
# In specific file mode, this is automatically set to the configured file
# In discovery mode, you can select a file from all_db_files or set it manually

if USE_SPECIFIC_FILE and target_db_file:
    selected_db_file = target_db_file
    print(f"Selected file (from configuration): {selected_db_file.name}")
    print(f"Path: {selected_db_file}")
elif all_db_files:
    # Default to first file found, but you can change this
    selected_db_file = all_db_files[0]['file_path']
    print(f"Selected file (first from discovery): {selected_db_file.name}")
    print(f"Farm: {all_db_files[0]['farm_name']}, Coordinate: {all_db_files[0]['coordinate']}")
    print(f"Path: {selected_db_file}")
    print(f"\nTo select a different file, set:")
    print(f"  selected_db_file = all_db_files[N]['file_path']  # where N is the index")
    print(f"\nAvailable files:")
    for i, file_info in enumerate(all_db_files):
        marker = " <-- SELECTED" if file_info['file_path'] == selected_db_file else ""
        print(f"  [{i}] {file_info['file_name']} ({file_info['farm_name']}/{file_info['coordinate']}){marker}")
else:
    selected_db_file = None
    print("No database file available. Please check your configuration.")

No database file available. Please check your configuration.


In [51]:
# Summary statistics
if db_files_by_location:
    print("=" * 80)
    print("Summary by Location")
    print("=" * 80)
    
    # Group by farm
    farms = {}
    for location_key, location_data in db_files_by_location.items():
        farm_name = location_data['farm_name']
        if farm_name not in farms:
            farms[farm_name] = {
                'coordinates': [],
                'total_files': 0
            }
        farms[farm_name]['coordinates'].append(location_data['coordinate'])
        farms[farm_name]['total_files'] += len(location_data['db_files'])
    
    print(f"\nTotal Farms: {len(farms)}")
    print(f"Total Coordinates: {len(db_files_by_location)}")
    print(f"Total Database Files: {total_files}\n")
    
    print("Breakdown by Farm:")
    print("-" * 80)
    for farm_name, farm_data in sorted(farms.items()):
        print(f"\n{farm_name}:")
        print(f"  Coordinates: {len(farm_data['coordinates'])}")
        print(f"  Database Files: {farm_data['total_files']}")
        print(f"  Coordinate List: {', '.join(sorted(farm_data['coordinates']))}")
else:
    print("No database files found.")

No database files found.


## Access Database Files

Use the `all_db_files` list to access all database files with their metadata, or iterate through `db_files_by_location` to process by location.

## Database Inspection Functions

Helper functions to inspect database structure and contents.

In [52]:
def inspect_database(db_path: Path) -> Dict[str, Any]:
    """
    Inspect an APSIM database file and return information about its structure.
    
    Args:
        db_path: Path to the SQLite database file
    
    Returns:
        Dictionary containing database metadata and table information
    """
    if not db_path.exists():
        return {"error": f"Database file not found: {db_path}"}
    
    conn = sqlite3.connect(str(db_path))
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    # Get all tables
    cursor.execute("""
        SELECT name FROM sqlite_master 
        WHERE type='table' 
        ORDER BY name
    """)
    tables = [row[0] for row in cursor.fetchall()]
    
    db_info = {
        "database_file": str(db_path),
        "tables": {},
        "has_report": "Report" in tables,
        "has_daily": "Daily" in tables
    }
    
    # Get schema and row counts for each table
    for table_name in tables:
        cursor.execute(f"SELECT COUNT(*) as count FROM {table_name}")
        row_count = cursor.fetchone()["count"]
        
        # Get column information
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = []
        for col in cursor.fetchall():
            columns.append({
                "name": col[1],
                "type": col[2],
                "not_null": bool(col[3]),
                "default": col[4],
                "primary_key": bool(col[5])
            })
        
        db_info["tables"][table_name] = {
            "row_count": row_count,
            "columns": columns,
            "column_names": [col["name"] for col in columns]
        }
    
    conn.close()
    return db_info


def print_database_summary(db_info: Dict[str, Any]) -> None:
    """Print a formatted summary of database information."""
    if "error" in db_info:
        print(f"Error: {db_info['error']}")
        return
    
    print("=" * 80)
    print(f"Database: {Path(db_info['database_file']).name}")
    print("=" * 80)
    print(f"\nTables found: {len(db_info['tables'])}")
    print(f"Has Report table: {db_info['has_report']}")
    print(f"Has Daily table: {db_info['has_daily']}\n")
    
    for table_name, table_data in db_info["tables"].items():
        print(f"📊 {table_name}")
        print(f"   Rows: {table_data['row_count']:,}")
        print(f"   Columns ({len(table_data['columns'])}): {', '.join(table_data['column_names'][:10])}")
        if len(table_data['column_names']) > 10:
            print(f"   ... and {len(table_data['column_names']) - 10} more")
        print()


print("✓ Database inspection functions loaded")

✓ Database inspection functions loaded


## Inspect Selected Database File

Inspect the currently selected database file (from configuration or selection).

In [53]:
# Inspect the selected database file
if 'selected_db_file' in globals() and selected_db_file and selected_db_file.exists():
    # Find file info if available
    file_info = None
    for info in all_db_files:
        if info['file_path'] == selected_db_file:
            file_info = info
            break
    
    if file_info:
        print(f"Inspecting: {selected_db_file.name}")
        print(f"Location: {file_info['farm_name']}/{file_info['coordinate']}\n")
    else:
        print(f"Inspecting: {selected_db_file.name}\n")
    
    db_info = inspect_database(selected_db_file)
    print_database_summary(db_info)
    
    # Check if Report and Daily tables have expected columns
    if db_info['has_report']:
        report_cols = db_info['tables']['Report']['column_names']
        # Note: Report table uses "Yield" not "DryYield"
        expected_report_cols = ['Year', 'Yield', 'SowingDate', 'HarvestDate', 'CropSurvived']
        missing_cols = [col for col in expected_report_cols if col not in report_cols]
        if missing_cols:
            print(f"⚠ Note: Some expected Report columns not found: {missing_cols}")
        else:
            print("✓ Report table contains expected columns")
    
    if db_info['has_daily']:
        daily_cols = db_info['tables']['Daily']['column_names']
        # Note: Daily table uses "RadiationsFactor" (with 's') not "RadiationFactor"
        expected_daily_cols = ['Date', 'Year', 'DryYield', 'RadiationsFactor', 'WaterFactor']
        missing_cols = [col for col in expected_daily_cols if col not in daily_cols]
        if missing_cols:
            print(f"⚠ Note: Some expected Daily columns not found: {missing_cols}")
        else:
            print("✓ Daily table contains expected columns")
else:
    print("No database file selected or file not found.")
    print("Please check your configuration or select a file from all_db_files.")

No database file selected or file not found.
Please check your configuration or select a file from all_db_files.


## Database to CSV/Excel Conversion

Convert the selected database file to CSV/Excel format. Creates an Excel file (.xlsx) with multiple sheets (one per table), or individual CSV files for each table.

In [54]:
def convert_db_to_excel(db_path: Path, output_path: Path = None, include_empty_tables: bool = False) -> Path:
    """
    Convert an APSIM SQLite database to an Excel file with multiple sheets.
    
    Args:
        db_path: Path to the SQLite database file
        output_path: Path for the output Excel file (default: same folder as db file, with .xlsx extension)
        include_empty_tables: Whether to include tables with 0 rows
    
    Returns:
        Path to the created Excel file
    """
    if not db_path.exists():
        raise FileNotFoundError(f"Database file not found: {db_path}")
    
    # Set default output path if not provided
    if output_path is None:
        output_path = db_path.parent / f"{db_path.stem}.xlsx"
    
    # Connect to database
    conn = sqlite3.connect(str(db_path))
    
    # Get all tables
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    tables = [row[0] for row in cursor.fetchall()]
    
    print(f"Converting database: {db_path.name}")
    print(f"Output file: {output_path.name}")
    print(f"Found {len(tables)} table(s)\n")
    
    # Create Excel writer
    with pd.ExcelWriter(str(output_path), engine='openpyxl') as writer:
        exported_tables = 0
        
        for table_name in tables:
            # Get row count
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cursor.fetchone()[0]
            
            if row_count == 0 and not include_empty_tables:
                print(f"  ⏭ Skipping {table_name} (empty table)")
                continue
            
            # Read table into DataFrame
            try:
                df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
                
                # Clean sheet name (Excel has limitations on sheet names)
                sheet_name = table_name[:31]  # Excel sheet name limit is 31 characters
                sheet_name = sheet_name.replace('/', '_').replace('\\', '_').replace('?', '_').replace('*', '_')
                sheet_name = sheet_name.replace('[', '').replace(']', '')
                
                # Write to Excel sheet
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                exported_tables += 1
                print(f"  ✓ Exported {table_name} ({row_count:,} rows) -> Sheet: {sheet_name}")
                
            except Exception as e:
                print(f"  ✗ Error exporting {table_name}: {e}")
    
    conn.close()
    
    print(f"\n✓ Conversion complete!")
    print(f"  Exported {exported_tables} table(s) to {output_path.name}")
    print(f"  File location: {output_path}")
    
    return output_path


def convert_db_to_csv_files(db_path: Path, output_dir: Path = None, include_empty_tables: bool = False) -> List[Path]:
    """
    Convert an APSIM SQLite database to multiple CSV files (one per table).
    
    Args:
        db_path: Path to the SQLite database file
        output_dir: Directory for CSV files (default: creates a folder named after the db file in the same directory)
        include_empty_tables: Whether to include tables with 0 rows
    
    Returns:
        List of paths to created CSV files
    """
    if not db_path.exists():
        raise FileNotFoundError(f"Database file not found: {db_path}")
    
    # Set default output directory if not provided
    if output_dir is None:
        # Create a folder named after the database file (without .db extension)
        output_dir = db_path.parent / db_path.stem
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Connect to database
    conn = sqlite3.connect(str(db_path))
    
    # Get all tables
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
    tables = [row[0] for row in cursor.fetchall()]
    
    print(f"Converting database: {db_path.name}")
    print(f"Output directory: {output_dir}")
    print(f"Found {len(tables)} table(s)\n")
    
    csv_files = []
    
    for table_name in tables:
        # Skip metadata tables (tables starting with underscore), except _InitialConditions
        # This skips: _Checkpoints, _Simulations, _Messages, _Units, etc.
        # Only _InitialConditions is included (and renamed to Parameters)
        if table_name.startswith('_') and table_name != '_InitialConditions':
            print(f"  ⏭ Skipping {table_name} (metadata table)")
            continue
        
        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        
        if row_count == 0 and not include_empty_tables:
            print(f"  ⏭ Skipping {table_name} (empty table)")
            continue
        
        # Read table into DataFrame
        try:
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            
            # Format Date and SowingDate columns for Daily table (remove time portion)
            if table_name == 'Daily':
                try:
                    # Convert Date to datetime if it exists (date-only, no time component)
                    if 'Date' in df.columns:
                        try:
                            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                            # Normalize to remove time component (keeps date-only)
                            df['Date'] = df['Date'].dt.normalize()
                        except Exception as e:
                            print(f"    ⚠ Warning: Could not format Date column: {e}")
                    
                    # Convert SowingDate to datetime if it exists (date-only, no time component)
                    if 'SowingDate' in df.columns:
                        try:
                            df['SowingDate'] = pd.to_datetime(df['SowingDate'], errors='coerce')
                            # Normalize to remove time component (keeps date-only)
                            df['SowingDate'] = df['SowingDate'].dt.normalize()
                        except Exception as e:
                            print(f"    ⚠ Warning: Could not format SowingDate column: {e}")
                    
                    # Fix Year column if it's 0 or missing - extract year from Date column
                    if 'Year' in df.columns and 'Date' in df.columns:
                        try:
                            # Extract year from Date column where Year is 0, missing, or invalid
                            # Only proceed if Date column has valid datetime values
                            if df['Date'].notna().any():
                                date_years = df['Date'].dt.year
                                # Update Year column where it's 0, NaN, or less than 1900, and Date is valid
                                mask = ((df['Year'] == 0) | (df['Year'].isna()) | (df['Year'] < 1900)) & df['Date'].notna()
                                df.loc[mask, 'Year'] = date_years[mask]
                        except Exception as e:
                            print(f"    ⚠ Warning: Could not fix Year column: {e}")
                except Exception as e:
                    print(f"    ⚠ Warning: Error in date formatting for {table_name}: {e}")
            
            # Create CSV filename (rename _InitialConditions to Parameters)
            display_name = "Parameters" if table_name == "_InitialConditions" else table_name
            csv_filename = f"{db_path.stem}_{display_name}.csv"
            csv_path = output_dir / csv_filename
            
            # Write to CSV
            df.to_csv(csv_path, index=False, encoding='utf-8')
            csv_files.append(csv_path)
            print(f"  ✓ Exported {display_name} ({row_count:,} rows) -> {csv_filename}")
            
        except Exception as e:
            print(f"  ✗ Error exporting {table_name}: {e}")
    
    conn.close()
    
    print(f"\n✓ Conversion complete!")
    print(f"  Exported {len(csv_files)} CSV file(s) to {output_dir}")
    
    return csv_files


print("✓ Database conversion functions loaded")

✓ Database conversion functions loaded


## Convert Selected Database to CSV

Convert the selected database file to CSV format. Creates multiple `.csv` files, one file per table.

**Output Location**: Files are saved in a **folder named after the database file** (without the `.db` extension), created in the same directory as the database file.

**Note**: Requires `pandas` package. Install with:
```bash
pip install pandas
```

**CSV Format**:
- Creates multiple `.csv` files, one file per table
- File names: `{database_name}_{table_name}.csv`
- Location: A folder named `{database_name}` (without `.db`) in the same directory as the database file
- Example: For `ClimAdapt_Wheat_neg31.45_117.55_past.db`, CSV files are saved in `ClimAdapt_Wheat_neg31.45_117.55_past/` folder:
  - `ClimAdapt_Wheat_neg31.45_117.55_past/ClimAdapt_Wheat_neg31.45_117.55_past_Daily.csv`
  - `ClimAdapt_Wheat_neg31.45_117.55_past/ClimAdapt_Wheat_neg31.45_117.55_past_Report.csv`
  - etc.


In [55]:
# Conversion configuration
INCLUDE_EMPTY_TABLES = False  # Set to True to include tables with 0 rows

# Convert the selected database file to CSV format
# Output files will be saved in a folder named after the database file (without .db extension)
if 'selected_db_file' in globals() and selected_db_file and selected_db_file.exists():
    try:
        # Output folder will be created: {database_file_name}/ (without .db extension)
        output_folder = selected_db_file.parent / selected_db_file.stem
        
        # Convert to multiple CSV files
        # Output will be saved in a folder named after the database file
        csv_files = convert_db_to_csv_files(
            selected_db_file,
            output_dir=None,  # None = create folder named after db file
            include_empty_tables=INCLUDE_EMPTY_TABLES
        )
        print(f"\n{'='*80}")
        print(f"✓ CSV files created successfully!")
        print(f"  Created {len(csv_files)} CSV file(s):")
        for csv_file in csv_files:
            print(f"    - {csv_file.name}")
        print(f"  Saved in: {output_folder}")
        print(f"{'='*80}")
            
    except ImportError as e:
        print(f"Error: Missing required package. {e}")
        print("\nPlease install required package:")
        print("  pip install pandas")
    except Exception as e:
        print(f"Error during conversion: {e}")
        import traceback
        traceback.print_exc()
else:
    print("No database file selected or file not found.")
    print("Please check your configuration or select a file from all_db_files.")

No database file selected or file not found.
Please check your configuration or select a file from all_db_files.
